## Data Parsing

### Data for api_details table

In [ ]:
api_details = {}
import mysql.connector

# Connect to MySQL
cnx = mysql.connector.connect(
    host='xxx',
    user='xxx',
    password='xxx',
    database='xxx'
)

cursor = cnx.cursor()
select_query = "SELECT a1.base_url, a1.followers, a1.resources, a1.ratings_votes, a1.documentation, a2.pageCount, a2.pageComment, a2.discussionCount FROM aboutPage a1 JOIN discussionPage a2 ON a1.base_url = a2.base_url"
#select_query = "SELECT base_url FROM discussionPage WHERE base_url not in (select base_url from aboutPage)"

cursor.execute(select_query)
 
results = cursor.fetchall()

for result in results:
        base_url = result[0]
        base_url = base_url[:255]
        followers = result[1]
        resources = result[2]
        ratings_votes = result[3]
        documentation = result[4]
        pageCount = result[5]
        pageComment = result[6]
        discussionCount = result[7]
        if ratings_votes != '':
            rating_parts = ratings_votes.split(" - ")
            ratings = rating_parts[0].split(": ")[1]
            votes = rating_parts[1].split(": ")[1]
        api_details[base_url] = {
            'followers': int(followers.replace(',', '')),
            'resources': resources,
            'ratings': float(ratings),
            'votes': int(votes),
            'documentation': int(documentation),
            'discussion_page_count':int(pageCount),
            'pageComment':int(pageComment),
            'discussionCount':int(discussionCount)
        }
cnx.close()   

In [ ]:
import mysql.connector

# Connect to MySQL
cnx = mysql.connector.connect(
    host='xxx',
    user='xxx',
    password='xxx',
    database='xxx'
)

cursor = cnx.cursor()

# Prepare the SQL statement for the insert operation
insert_api_details = "INSERT INTO api_details (base_url, followers, resources, ratings, votes, documentation, discussion_page_count, pageComment, discussionCount) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"

# Iterate over the items in the api_details dictionary
for base_url, details in api_details.items():
    followers = int(details['followers'])
    resources = details['resources']
    ratings = float(details['ratings'])
    votes =  int(details['votes'])
    documentation =  int(details['documentation'])
    discussion_page_count = int(details['discussion_page_count'])
    pageComment =  int(details['pageComment'])
    discussionCount =  int(details['discussionCount'])
    

    # Check if URL already exists in the database
    select_query = "SELECT base_url FROM api_details WHERE base_url = %s"
    cursor.execute(select_query, (base_url,))
    existing_url = cursor.fetchone()

    if existing_url:
        continue
    else:
        # Execute the insert operation for each URL and details
        cursor.execute(insert_api_details, (base_url, followers, resources, ratings, votes, documentation, discussion_page_count, pageComment, discussionCount))

# Commit the changes and close the connection
cnx.commit()
cnx.close()

### Data for api_info table

In [24]:
import re
import mysql.connector
from urllib.parse import urlparse

api_details = {}
api_info ={}
# Connect to MySQL
cnx = mysql.connector.connect(
    host='xxx',
    user='xxx',
    password='xxx',
    database='xxx'
)

cursor = cnx.cursor()
select_query = "SELECT base_url, top_des, endpoint_des, api_url, hostname from endpointPage"

cursor.execute(select_query)
 
results = cursor.fetchall()

for result in results:
    base_url = result[0]
    print(base_url)
    top_des = result[1]
    endpoint_des = result[2]
    api_url = result[3]
    hostname = result[4]
    
    if hostname =='':
        parsed_url = urlparse(api_url)
        hostname = parsed_url.hostname
        
    # Splitting the string based on '|' and removing leading/trailing spaces
    split_data = top_des.split('|')
    split_data = [item.strip() for item in split_data]
    # Extracting the different values
    job_search = split_data[0]  # "JobSearchFREEByAva"
#     last_updated = split_data[1]  # "Updated a month ago"
#     print(last_updated)
    
    # Check if split_data has at least 2 elements before accessing index 1
    if len(split_data) > 1:
        last_updated = split_data[1]
    else:
        last_updated = ''  # Set a default value if split_data doesn't have enough elements


    jobs = ''
    print(split_data)
    try:
        category = split_data[2].split('Popularity')[0].strip()  # "Jobs"
        category = category[1:] if category.startswith('-') else category  # Removing the leading '-' if present
        popularity_data = split_data[2].split('Popularity') if len(split_data) > 2 else []
        popularity_value = popularity_data[1].split('/')[0].strip() if len(popularity_data) > 1 else 0.0
        popularity = float('.'.join(popularity_value.split()[-1].split('.')[:2])) if popularity_value else 0.0

        latency_data = split_data[2].split('Latency') if len(split_data) > 2 else []
        latency = int(latency_data[1].split('ms')[0].strip().replace(',', '')) if len(latency_data) > 1 else 0

        service_level_data = split_data[2].split('Service Level') if len(split_data) > 2 else []
        service_level = int(service_level_data[1].split('%')[0]) if len(service_level_data) > 1 else 0
        
        

    except IndexError:
        # Handle the case where the string does not match the expected pattern
        print(base_url,"Error: Unable to extract values from the string.")
 
    
    api_info[base_url] = {
    'category': category,
    'last_updated': last_updated,
    'endpoint_des': endpoint_des,
    'popularity': popularity,
    'latency': latency,
    'service_level': service_level,
    'api_url': api_url,
    'hostname': hostname
        
}
cnx.close()

https://rapidapi.com/musex/api/tes130/
['TESFREEBymusex', 'Updated 8ヶ月前', 'CybersecurityPopularity0.3 / 10Latency431msService Level100%Health CheckN/A']
https://rapidapi.com/api-method-api-method-default/api/mitre-att-ck-framework/
['MITRE ATT&CK FrameworkFREEMIUMByAPI Method', 'Updated 2 months ago', 'CybersecurityPopularity8.3 / 10Latency822msService Level100%Health CheckN/A']
https://rapidapi.com/srivishnuk90/api/spamsafe/
['SpamSafePAIDBySri Vishnu', 'Updated 4 months ago', 'CybersecurityHealth CheckN/A']
https://rapidapi.com/sujoyk211/api/true_caller-api/
['True_Caller APIFREEMIUMByDUPELICATE GAMERZ', 'Updated месяц назад', 'CybersecurityPopularity9 / 10Latency5,367msService Level100%Health Check100%']
https://rapidapi.com/GlyphDLP/api/glyph-dlp/
['Glyph DLPFREEByGlyphDLP', 'Updated 3 months ago', 'CybersecurityHealth CheckN/A']
https://rapidapi.com/fptlapwifivn/api/demo-project51070/
['👋 Demo ProjectFREEByLắp WiFi FPT', 'Updated hace 4 meses', 'CybersecurityHealth CheckN/A']
http

['IP InsightFREEMIUMByAnthony Smith', 'Updated 8 महीने पहले', 'LocationHealth CheckN/A']
https://rapidapi.com/stevage/api/which-postcode/
['Which postcodeFREEMIUMBySteve Bennett', 'Updated 2 years ago', 'LocationHealth CheckN/A']
https://rapidapi.com/natkapral/api/ip-geo-location/
['IP Geo LocationFREEMIUMVerified✓ByGeo API', 'Updated a year ago', 'LocationPopularity9.9 / 10Latency97msService Level100%Health CheckN/A']
https://rapidapi.com/loribatta95/api/apiitaly/
['APIItalyFREEMIUMByLorenzo Battanello', 'Updated a year ago', 'LocationHealth CheckN/A']
https://rapidapi.com/nexxai/api/simple-time-zone-lookup/
['Simple Time Zone LookupFREEMIUMBynexxai', 'Updated 6 months ago', 'LocationHealth CheckN/A']
https://rapidapi.com/apichest/api/ip-geo-location-and-ip-reputation/
['IP Geo Location and IP ReputationFREEMIUMByapichest', 'Updated 2 anni fa', 'LocationPopularity0.3 / 10Latency18msService Level0%Health CheckN/A']
https://rapidapi.com/NovusAPI/api/nearby-tesla-superchargers/
['Nearby 

https://rapidapi.com/aptitudeapps/api/noaa-tides/
['API-BASKETBALLFREEMIUMVerified✓ByAPI-SPORTS', 'Updated 8 months ago', 'SportsPopularity9.9 / 10Latency314msService Level100%Health CheckN/A']
https://rapidapi.com/mahmoudahmed8081/api/climate-news27/
['API-BASKETBALLFREEMIUMVerified✓ByAPI-SPORTS', 'Updated 8 months ago', 'SportsPopularity9.9 / 10Latency314msService Level100%Health CheckN/A']
https://rapidapi.com/anttickner/api/met-office-historical-data/
['API-BASKETBALLFREEMIUMVerified✓ByAPI-SPORTS', 'Updated 8 months ago', 'SportsPopularity9.9 / 10Latency314msService Level100%Health CheckN/A']
https://rapidapi.com/isaiasponce12/api/climate13/
['API-BASKETBALLFREEMIUMVerified✓ByAPI-SPORTS', 'Updated 8 months ago', 'SportsPopularity9.9 / 10Latency314msService Level100%Health CheckN/A']
https://rapidapi.com/alvaromasilva/api/locationweatherapi/
['API-BASKETBALLFREEMIUMVerified✓ByAPI-SPORTS', 'Updated 8 months ago', 'SportsPopularity9.9 / 10Latency314msService Level100%Health CheckN/A']

In [29]:
import re
import mysql.connector

# Connect to MySQL
cnx = mysql.connector.connect(
    host='xxx',
    user='xxx',
    password='xxx',
    database='xxx'
)
cursor = cnx.cursor()

insert_api_info = "INSERT INTO api_info (base_url, category, last_updated, endpoint_des, popularity, latency, service_level, api_url, hostname) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"

# Iterate over the items in the api_details dictionary
for base_url, info in api_info.items():
    base_url = base_url[:255]
    category = info['category']
    last_updated = info['last_updated']
    endpoint_des = info['endpoint_des']
    popularity = info['popularity']
    latency = info['latency']
    service_level = info['service_level']
    api_url = info['api_url']
    hostname = info['hostname']

    # Check if URL already exists in the database
    select_query = "SELECT base_url FROM api_info WHERE base_url = %s"
    cursor.execute(select_query, (base_url,))
    existing_url = cursor.fetchone()

    if existing_url:
        print(f"URL {base_url} already exists in the database. Skipping insertion.")
    else:
        # Execute the insert operation for each URL and details
        cursor.execute(insert_api_info, (base_url, category, last_updated, endpoint_des, popularity, latency, service_level, api_url, hostname))
        
        # Commit the changes and close the connection
cnx.commit()
cnx.close()